In [13]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [14]:
port_sizes = [10, 100, 1000, 2_500, 10_000, 25_000, 40_000, 50_000, 100_000]
buffer_size_min_mb = 0.01
buffer_size_max_mb = 1024
num_points_to_plot = 250
buffer_size_var = np.linspace(
    buffer_size_min_mb, buffer_size_max_mb, num_points_to_plot
)


time_to_fill_buffer = [buffer_size_var * 8 / port_size for port_size in port_sizes]

In [15]:
fig = go.Figure()
for index in range(len(port_sizes)):
    if port_sizes[index] < 1_000:
        label = f"{port_sizes[index]} Mbps"
    elif int(port_sizes[index] / 1000) == port_sizes[index] / 1000:
        label = f"{int(port_sizes[index] / 1000)} Gbps"
    else:
        label = f"{port_sizes[index] / 1000} Gbps"
    fig.add_trace(
        go.Scatter(
            x=np.asarray(time_to_fill_buffer[index]),
            y=np.asarray(buffer_size_var),
            mode="lines",
            name=label,
        )
    )
fig.update_layout(
    xaxis_title="Time to fill buffer (s)",
    yaxis_title="Buffer size (MB)",
)
fig.update_xaxes(type="log")
fig.update_yaxes(
    type="log",
)
fig.show()

In [16]:
switch_metrics = pd.read_excel("Network Switch Performance.xlsx")
switch_metrics.head()
switch_metrics["Total Port Count"] = (
    switch_metrics["port 1 count"]
    + switch_metrics["port 2 count"]
    + switch_metrics["port 3 count"]
    + switch_metrics["port 4 count"]
)
switch_metrics["Expected Switching Capacity"] = (
    switch_metrics["port 1 count"] * switch_metrics["port 1 speed"]
    + switch_metrics["port 2 count"] * switch_metrics["port 2 speed"]
    + switch_metrics["port 3 count"] * switch_metrics["port 3 speed"]
    + switch_metrics["port 4 count"] * switch_metrics["port 4 speed"]
) / 1000
switch_metrics["Average Port Speed (Gbps)"] = (
    switch_metrics["Expected Switching Capacity"] / switch_metrics["Total Port Count"]
)
switch_metrics["Time to Fill Buffer (s)"] = (
    switch_metrics["Buffer Size (MB)"] * 8 / switch_metrics["Average Port Speed (Gbps)"] / 1000
)

In [17]:
fig2 = px.scatter(
    switch_metrics,
    x="Switching Capacity (Gbps)",
    y="Buffer Size (MB)",
    title="Buffer Size by Total Switching Capacity",
    color="Manufacturer",
    symbol="Series",
    log_x=True,
    log_y=True,
)
fig2.show()

In [18]:
fig3 = px.scatter(
    switch_metrics,
    x="Average Port Speed (Gbps)",
    y="Buffer Size (MB)",
    title="Buffer Size by Average Port Speed",
    color="Manufacturer",
    symbol="Series",
    log_x=True,
    log_y=True,
)
fig3.show()

In [19]:
fig4 = px.scatter(
    switch_metrics,
    x="Total Port Count",
    y="Buffer Size (MB)",
    title="Buffer Size by Number of Ports",
    color="Manufacturer",
    symbol="Series",
    log_x=True,
    log_y=True,
)
fig4.show()

In [24]:
fig5 = px.scatter(
    switch_metrics,
    x="Time to Fill Buffer (s)",
    y="Buffer Size (MB)",
    title="Buffer Size by Time to Fill Buffer",
    color="Manufacturer",
    size="Total Port Count",
    labels={
        "Time to Fill Buffer (s)": "Round Trip Time (s)",
    },    
    # symbol="Total Port Count",
    hover_data=["Model", "Switching Capacity (Gbps)"],
    log_x=True,
    log_y=True,
)
fig5.show()

In [28]:
fig6 = px.scatter(
    switch_metrics,
    x="Time to Fill Buffer (s)",
    y="Buffer Size (MB)",
    title="Network Switch Comparison",
    color="Manufacturer",
    labels={
        "Time to Fill Buffer (s)": "Round Trip Time (s)",
    },
    # size="Total Port Count",
    # symbol="Total Port Count",
    hover_data=["Model", "Switching Capacity (Gbps)"],
    log_x=True,
    log_y=True,
)
fig6.show()
fig6.write_html("network_switch_comparison.html")